In [9]:
# 📦 Imports
import pandas as pd
import numpy as np
import os

In [10]:
# 📂 Load dataset
data_path = "../data/processed/retail_sales_extended.csv"
df = pd.read_csv(data_path, parse_dates=["date"])

In [11]:
# 🧹 Clean up
df.drop(columns=["Unnamed: 0"], inplace=True)

In [12]:
# 🔁 Sort by item and date for lag feature consistency
df.sort_values(by=["item", "date"], inplace=True)

In [13]:
# 🧠 Add more lag features
for lag in [1, 3, 7]:
    df[f"sales_lag_{lag}"] = df.groupby("item")["units_sold"].shift(lag)

In [14]:
# 📈 Rolling average & trend (over 30 days)
df["sales_30d_avg"] = df.groupby("item")["units_sold"].transform(lambda x: x.rolling(30, min_periods=1).mean().shift(1))
df["sales_30d_trend"] = df["sales_30d_avg"] - df["sales_7d_avg"]

In [15]:
# 🗓️ Encode time-based features
df["day_of_week"] = df["date"].dt.dayofweek  # 0=Monday, 6=Sunday
df["month"] = df["date"].dt.month

In [16]:
# show first few rows
df.head()

,date,item,category,price,promotion,units_sold,sales_7d_avg,day_of_week,is_weekend,is_holiday,brand,discount,stock_available,stockout_flag,sales_lag_1,sales_lag_3,sales_lag_7,sales_30d_avg,sales_30d_trend,month
0,2023-10-01,Diaper,Baby Care,220,0,45,NaN,6,1,0,Pampers,0.12,51,0,NaN,NaN,NaN,NaN,NaN,10
1,2023-10-02,Diaper,Baby Care,200,0,51,45.0,0,0,0,Pampers,0.20,52,0,45.0,NaN,NaN,45.0,0.0,10
2,2023-10-03,Diaper,Baby Care,200,0,45,48.0,1,0,0,Pampers,0.20,51,0,51.0,NaN,NaN,48.0,0.0,10
3,2023-10-04,Diaper,Baby Care,200,0,55,47.0,2,0,1,Pampers,0.20,63,0,45.0,45.0,NaN,47.0,0.0,10
4,2023-10-05,Diaper,Baby Care,250,0,53,49.0,3,0,0,Pampers,0.00,57,0,55.0,51.0,NaN,49.0,0.0,10


In [17]:
# 🔤 Encode categorical features (Label Encoding or One-Hot Encoding as needed)
df["item"] = df["item"].astype("category").cat.codes
df["category"] = df["category"].astype("category").cat.codes
df["brand"] = df["brand"].astype("category").cat.codes

In [18]:
# show first few rows
df.head()

,date,item,category,price,promotion,units_sold,sales_7d_avg,day_of_week,is_weekend,is_holiday,brand,discount,stock_available,stockout_flag,sales_lag_1,sales_lag_3,sales_lag_7,sales_30d_avg,sales_30d_trend,month
0,2023-10-01,0,0,220,0,45,NaN,6,1,0,2,0.12,51,0,NaN,NaN,NaN,NaN,NaN,10
1,2023-10-02,0,0,200,0,51,45.0,0,0,0,2,0.20,52,0,45.0,NaN,NaN,45.0,0.0,10
2,2023-10-03,0,0,200,0,45,48.0,1,0,0,2,0.20,51,0,51.0,NaN,NaN,48.0,0.0,10
3,2023-10-04,0,0,200,0,55,47.0,2,0,1,2,0.20,63,0,45.0,45.0,NaN,47.0,0.0,10
4,2023-10-05,0,0,250,0,53,49.0,3,0,0,2,0.00,57,0,55.0,51.0,NaN,49.0,0.0,10


In [19]:
# 🧽 Handle missing values from lag features
df.fillna(0, inplace=True)

In [20]:
# show first few rows
df.head()

,date,item,category,price,promotion,units_sold,sales_7d_avg,day_of_week,is_weekend,is_holiday,brand,discount,stock_available,stockout_flag,sales_lag_1,sales_lag_3,sales_lag_7,sales_30d_avg,sales_30d_trend,month
0,2023-10-01,0,0,220,0,45,0.0,6,1,0,2,0.12,51,0,0.0,0.0,0.0,0.0,0.0,10
1,2023-10-02,0,0,200,0,51,45.0,0,0,0,2,0.20,52,0,45.0,0.0,0.0,45.0,0.0,10
2,2023-10-03,0,0,200,0,45,48.0,1,0,0,2,0.20,51,0,51.0,0.0,0.0,48.0,0.0,10
3,2023-10-04,0,0,200,0,55,47.0,2,0,1,2,0.20,63,0,45.0,45.0,0.0,47.0,0.0,10
4,2023-10-05,0,0,250,0,53,49.0,3,0,0,2,0.00,57,0,55.0,51.0,0.0,49.0,0.0,10


In [21]:
# 💾 Save engineered features
output_path = "../data/processed/retail_sales_features.csv"
df.to_csv(output_path, index=False)

print(f"✅ Feature engineering complete. Saved to {output_path}")

✅ Feature engineering complete. Saved to ../data/processed/retail_sales_features.csv
